In [1]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import multivariate_normal as mvn

from Gaussian import Gaussian

In [2]:
# Standard univariate Gaussian
test = Gaussian()
samp = test.sample(num_samp=10)
dens = test.log_p(samp)

print(test.mean)
print(test.cov)
print(test.chol)
print(samp)
print(dens)

[0.]
[[1.]]
[[1.]]
[[ 1.78669461]
 [ 0.19547627]
 [ 0.3552621 ]
 [ 1.83910995]
 [-1.40367381]
 [ 0.97199689]
 [-0.51387928]
 [-0.23439697]
 [-0.09618533]
 [ 0.71775747]]
[-2.51507736 -0.93804402 -0.98204411 -2.61010123 -1.90408862 -1.39132751
 -1.05097449 -0.9464095  -0.92356434 -1.17652643]


In [3]:
# Multivariate Gaussian
x = Gaussian(mean = np.ones(4), store="cov")
samp = x.sample(num_samp=3)
dens = x.log_p(samp)


print("mean:")
print(x.mean)
print("cov:")
print(x.cov)
print("chol:")
print(x.chol)
print("samp:")
print(samp)
print("dens:")
print(dens)

mean:
[1. 1. 1. 1.]
cov:
[[1. 0. 0. 0.]
 [0. 1. 0. 0.]
 [0. 0. 1. 0.]
 [0. 0. 0. 1.]]
chol:
[[1. 0. 0. 0.]
 [0. 1. 0. 0.]
 [0. 0. 1. 0.]
 [0. 0. 0. 1.]]
samp:
[[ 0.44102502  0.93437041  0.0856466   1.65491164]
 [ 0.77768939  2.46781746  1.14002705  0.15123346]
 [ 0.41905295 -0.34498954  2.15716576 -0.37302543]]
dens:
[-4.46660997 -5.14771529 -6.36111801]


In [11]:
# Test against scipy.
L = np.random.normal(size=(10,10))
C = L @ L.T
m = np.random.normal(size=10)
x_sp = mvn(mean=m, cov=C)

# From Cholesky factor.
x1 = Gaussian(mean=m, chol=L)

# From covariance.
x2 = Gaussian(mean=m, cov=C)

In [15]:
# Samples
n_samp = 10000
samp_sp = x_sp.rvs(size=n_samp)
samp_x1 = x1.sample(n_samp)
samp_x2 = x2.sample(n_samp)

AttributeError: 'Gaussian' object has no attribute 'mean'

In [14]:
samp_sp.shape

(10000, 10)